# Reinforcement Learning

**What is Reinforcement Learning**

Reinforcement learning (RL) is an area of machine learning concerned with how software agents ought to take actions in an environment in order to maximize some notion of cumulative reward. Reinforcement learning is one of three basic machine learning paradigms, alongside supervised learning and unsupervised learning. 

**Objective**

Basically, we will implement a simple game  (tic-tac-toe) to enhance the learning of the reinfocement learning algorithm.

**How we will implement**

To formulate this reinfocement learning problem, we will focus int three major components: **state, action and reward**.

Therefore, we will initialise a 3x3 board with zeros indicationg available positions and update positions with 1 if player 1 takes a move and -1 if player 2 takes a move. The action is what positions a  player can choose based on the current board state. Reward is between 0 and 1 and is only given at the end of the game.

For each state of the game, we will get the hash code of the current state of the board. To implement this, we'll use hash function, that is, we'll map the current board to a hash code to specify each board with its own code.

**Reference**

https://towardsdatascience.com/reinforcement-learning-implement-tictactoe-189582bea542

https://en.wikipedia.org/wiki/Reinforcement_learning


In [0]:
# We will follow this two sessions to implement an algorithm to
# make a machine learn to play tic-tac-toe through the reinforcement learning
# algorithm:
#
#   1) Train two agents (machine) to play against each other and save their
#   policy that was given by the reward at the end of each game 
#   2) Load the policy and make the agent to play against human



In [0]:
import numpy as np # import numpy to use a simple array to represent the board
import pickle

In [0]:
# Tic-Tac-Toe board rows and columns
BOARD_ROWS = 3
BOARD_COLS = 3

### Board State



1.   Act as both board and judger
2.   It has functions recording board state of both players and update state when either player takes an action
3. At the end of the game, its be able to judge what was the best play and give reward to the player that did it  




In [0]:
class State:
  
  # Its like cpp constructor but here we have a predefined method (__init__)
  def __init__(self, p1, p2):
    # Init the board with zeros (its a 3x3 array)
    self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
    self.p1 = p1
    self.p2 = p2
    self.isEnd = False
    self.boardHash = None # hash of the board is not initialized because we will use getHash function to do it
    #init p1 plays first
    self.playerSymbol = 1

  # Hashes the current board states, so that ir can be stored in 
  # the state-value dictionary
  def getHash(self):
    # reshape the 3x3 array to a 1x9 vector (to get the board hash)
    self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
    return self.boardHash

  # Checks sum of rows, columns and diagonals, and return 1 if p1 
  # wins, -1 if p2 wins, 0 if draw and None if the game is not yed ended
  # At the end of the game, 1 is rewarded to winner and 0 to loser. 
  # We can possibly modify the reward according to the game
  def winner(self):
    # row
    for i in range(BOARD_ROWS):
      # checks if has a row with p1
      if sum(self.board[i, :]) == 3:
        self.isEnd = True
        return 1
      # checks if has a row with p2
      if sum(self.board[i, :]) == -3:
        self.isEnd = True
        return -1
    
    # col
    for i in range(BOARD_COLS):
      # check if has a col with p1
      if sum(self.board[:, i]) == 3:
        self.isEnd = True
        return 1
      # checks if has a col with p2
      if sum(self.board[:, i]) == -3:
        self.isEnd = True
        return -1

    #diagonal
    diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
    diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
    diag_sum = max(abs(diag_sum1), abs(diag_sum2))

    # If have a triple of player 1 (or 2) in the diagonal
    if diag_sum == 3:
      self.isEnd = True
      # If the player 1 won
      if diag_sum1 == 3 or diag_sum2 == 3:
        return 1
      # If the player 2 won
      else:
        return -1

    # tie
    # no available positions
    if len(self.availablePositions()) == 0:
      self.isEnd = True
      return 0

    # not end
    self.isEnd = False
    return None
 
  # Checks for possible positions to play
  # Returns a tuple to be used in the updateState method
  def availablePositions(self):
    positions = [] # initialize a list of tuples (ordered pairs)
    for i in range(BOARD_ROWS):
      for j in range(BOARD_COLS):
        # checks if its a available position
        if self.board[i, j] == 0:
          positions.append((i, j)) # need to be a tuple
    return positions

  # Update the state of the board according to the players move
  def updateState(self, position):
    self.board[position] = self.playerSymbol
    # switch to another player
    self.playerSymbol = -1 if self.playerSymbol == 1 else 1

  # only when the game ends
  def giveReward(self):
    result = self.winner()
    # backpropagate reward
    # checks if the p1 won
    if result == 1:
      self.p1.feedReward(1)
      self.p2.feedReward(0)
    # checks ithe p2 won
    elif result == -1:
      self.p1.feedReward(0)
      self.p2.feedReward(1)
    else:
      # if tie
      self.p1.feedReward(0.1)
      self.p2.feedReward(0.5)


  def reset(self):
    self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
    self.boardHash = None
    self.isEnd = False
    self.playerSymbol = 1



  # Training the algorithm:
  # During training, the process for each player is:
  #   - Look for available positions
  #   - Choose action
  #   - Update board state and add the action to players states
  #   - Judge if reach the end of the game and give reward accordingly
  def play(self, rounds=100):
    for i in range(rounds):
      if i % 1000 == 0:
        print("Rounds {}".format(i))
      while not self.isEnd:
        # Player 1
        positions = self.availablePositions()
        p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
        # take action and update board state
        self.updateState(p1_action)
        board_hash = self.getHash()
        self.p1.addState(board_hash)
        # check board status if is end

        win = self.winner()
        if win is not None:
          #self.showBoard()
          #ended with p1 either win or draw
          self.giveReward()
          self.p1.reset()
          self.p2.reset()
          self.reset()
          break

        else:
          #Player 2
          positions = self.availablePositions()
          p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
          self.updateState(p2_action)
          board_hash = self.getHash()
          self.p2.addState(board_hash)

          win = self.winner()
          if win is not None:
            #self.showBoard()
            #ended with p2 either win or draw
            self.giveReward()
            self.p1.reset()
            self.p2.reset() 
            self.reset()
            break

  # play against human
  def play2(self):
    while not self.isEnd:
      # Player 1
      positions = self.availablePositions()
      p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
      # take action and upate board state
      self.updateState(p1_action)
      self.showBoard()
      # check board status if it is end
      win = self.winner()
      if win is not None:
        if win == 1:
          print(self.p1.name, "wins!")
        else:
          print("tie!")
        self.reset()
        break

      else:
        # Player 2
        positions = self.availablePositions()
        p2_action = self.p2.chooseAction(positions)

        self.updateState(p2_action)
        self.showBoard()
        win = self.winner()
        if win is not None:
          if win == -1:
            print(self.p2.name, "wins!")
          else:
            print("tie!")
          self.reset()
          break


  def showBoard(self):
    # p1: x  p2: o
    for i in range(0, BOARD_ROWS):
      print('-------------')
      out = '| '
      for j in range(0, BOARD_COLS):
        if self.board[i, j] == 1:
          token = 'x'
        if self.board[i, j] == -1:
          token = 'o'
        if self.board[i, j] == 0:
          token = ' '
        out += token + ' | '
      print(out)
    print('-------------')




### Player

* Represent our agent
* The player is able to:
  1.   Choose actions base on current estimation of the states 
  2. Record all states of the game
  3. Update states-value estimation after each game
  4. Save and load the policy 
* We keep track of all positions the player's been during each game in a list and update the correspondig states in a dictonary
* We use gama-greedy method to balance between exploration and exploitation     




In [0]:
class Player:
  # init a dict storing state-value pair and update
  # the estimates at the end of each game

  # exp_rate means gamma = 0.3, in other words 70% of the
  # time our agent will take greedy action, witch is choosing
  # action based on current estimation of states-value, and 30%
  # of the time our agent will take random action
  def __init__(self, name, exp_rate=0.3):
    self.name = name
    self.states = [] # record all positions taken
    self.lr = 0.2
    self.exp_rate = exp_rate
    self.decay_gamma = 0.9
    self.states_value = {} # state -> value

  def getHash(self, board):
    boardHash = str(board.reshape(BOARD_COLS * BOARD_ROWS))
    return boardHash

  def chooseAction(self, positions, current_board, symbol):
    # generates a random number and compares it with exp_rate
    if np.random.uniform(0, 1) <= self.exp_rate:
      # take random action according to the positions
      idx = np.random.choice(len(positions))
      action = positions[idx]
    else:
      value_max = -999
      for p in positions:
        next_board = current_board.copy()
        next_board[p] = symbol
        next_boardHash = self.getHash(next_board)
        # we store the hash of the board into state-value dict,
        # and while axploitation, we hash de next board state and
        # choose the action that returns the maximum value of the next state
        # if the get method (dict) didnt found the especific value, returns None
        value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
        # print("value", value)
        if value >= value_max:
          value_max = value
          action = p
    # print("{} takes action {}".format(self.name, action))
    return action

  # append hash state
  def addState(self, state):
    self.states.append(state)

  # At the end of the game, backpropagate and update states-value
  # To update states-value we will apply the following formula:
  # V(St) <-> V(St) + x [V(St+1) - V(St)]
  # that tells us that the value of state t equals the value of
  # state t adding the difference between the value of the state
  # t+1 and the value o the state t, which is multiplied by a 
  # learning rate x (Given the reward of intermediate state is 0)
  # We will update teh current value based on our latest observation
  def feedReward(self, reward):
    # the positions of each game is sotred in self.states and when
    # the agent reach the end of the game, the estimates are updates
    # in reversed fashion
    for st in reversed(self.states):
      # if st doesnt exists
      if self.states_value.get(st) is None:
        self.states_value[st] = 0
      self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
      reward = self.states_value[st]


  # ??????????
  def reset(self):
    self.states = []

  # Save policy to play with a human player
  # this dunction is called at the end of the training
  def savePolicy(self):
    fw = open('policy_' + str(self.name), 'wb')
    pickle.dump(self.states_value, fw)
    fw.close()

  def loadPolicy(self, file):
    fr = open(file, 'rb')
    self.states_value = pickle.load(fr)
    fr.close()

  # ?????????
  


  

### Human Player



*   Represents the player effectively
*   This class includes only 1 usable function chooseAction which requires us to input the board position we hope to take. And we also need to modify a bit on the play function inside State class 



In [0]:
class HumanPlayer:
  def __init__(self, name):
    self.name = name

  def chooseAction(self, positions):
    while True:
      row = int(input("Input your action row: "))
      col = int(input("Input your action col: "))
      action = (row, col)
      if action in positions:
        return action

  # append a hash state
  def addState(self, state):
    pass

  # at the end of the game, backpropagate and update states value
  def feedReward(self, reward):
    pass

  def reset(self):
    pass

### Training

In [13]:
# Training the algorithm
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)
print("training...")
st.play(10000)

training...
Rounds 0
Rounds 1000
Rounds 2000
Rounds 3000
Rounds 4000
Rounds 5000
Rounds 6000
Rounds 7000
Rounds 8000
Rounds 9000


In [0]:
p1.savePolicy()
p2.savePolicy()

In [0]:
p1.loadPolicy("policy_p1")

###Human vs Computer

In [17]:
p1 = Player("computer", exp_rate=0)
p1.loadPolicy("policy_p1")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play2()

-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
| x |   |   | 
-------------
Input your action row: 0
Input your action col: 2
-------------
|   |   | o | 
-------------
|   |   |   | 
-------------
| x |   |   | 
-------------
-------------
|   |   | o | 
-------------
|   |   |   | 
-------------
| x |   | x | 
-------------
Input your action row: 2
Input your action col: 1
-------------
|   |   | o | 
-------------
|   |   |   | 
-------------
| x | o | x | 
-------------
-------------
| x |   | o | 
-------------
|   |   |   | 
-------------
| x | o | x | 
-------------
Input your action row: 1
Input your action col: 1
-------------
| x |   | o | 
-------------
|   | o |   | 
-------------
| x | o | x | 
-------------
-------------
| x |   | o | 
-------------
| x | o |   | 
-------------
| x | o | x | 
-------------
computer wins!
